# Web Server Log Analysis  

## Overview
This assessment involves analyzing the Calgary HTTP dataset, which contains approximately one year's worth of HTTP requests to the University of Calgary's Computer Science web server. You'll work with real-world web server log data to extract meaningful insights and demonstrate your Python data analysis skills.

## Part 1: Data Loading and Cleaning

In [3]:
import urllib.request
import gzip
import re
import os
import pandas as pd
from datetime import datetime
from warnings import filterwarnings
filterwarnings('ignore')

In [4]:
# Download .gz from FTP

ftp_url = "ftp://ita.ee.lbl.gov/traces/calgary_access_log.gz"
local_filename = "calgary_access_log.gz"

try:
    print("Downloading file...")
    urllib.request.urlretrieve(ftp_url, local_filename)
    print("File downloaded successfully!")
except Exception as e:
    print("Error downloading file:", e)

Error downloading file: <urlopen error ftp error: error_reply('229 Extended Passive Mode Entered (|||25028|).')>


In [5]:
# Read .gz file line by line

log_lines = []

try:
    with gzip.open(local_filename, 'rt', encoding='latin1') as f:
        for line in f:
            log_lines.append(line.strip())
    print("File read successfully with", len(log_lines), "log entries.")
except Exception as e:
    print("Error reading the file:", e)

File read successfully with 726739 log entries.


In [6]:
# Function to parse each log line

def parse_log_line(line):
    pattern = (
        r'(?P<host>\S+) \S+ \S+ '
        r'\[(?P<timestamp>[^\]]+)\] '
        r'"(?P<request>[^"]+)" '
        r'(?P<status>\d{3}) (?P<size>\S+)'
    )
    match = re.match(pattern, line)
    if not match:
        return None
    
    data = match.groupdict()

    # Handle request (method, URL, protocol)
    parts = data['request'].split()
    if len(parts) == 3:
        data['method'], data['url'], data['protocol'] = parts
    else:
        data['method'], data['url'], data['protocol'] = None, None, None

    # Parse timestamp
    try:
        data['timestamp'] = datetime.strptime(data['timestamp'], "%d/%b/%Y:%H:%M:%S %z")
    except:
        data['timestamp'] = None

    # Convert size
    data['size'] = int(data['size']) if data['size'].isdigit() else None

    # Extract file extension
    if data['url']:
        _, ext = os.path.splitext(data['url'])
        data['file_ext'] = ext if ext else None
    else:
        data['file_ext'] = None

    return data

In [7]:
# Apply parsing and clean malformed entries

parsed_data = [parse_log_line(line) for line in log_lines]
parsed_data = [entry for entry in parsed_data if entry is not None]

print("Parsed valid entries:", len(parsed_data))

Parsed valid entries: 724836


In [8]:
# Load into DataFrame and clean types
df = pd.DataFrame(parsed_data)

# Drop rows with missing timestamps or URLs
df_clean = df.dropna(subset=['timestamp', 'url'])

# Convert to UTC and making timezone
df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'], errors='coerce', utc=True)
df_clean['timestamp'] = df_clean['timestamp'].dt.tz_localize(None)

# Convert to the appropriate columns
df_clean['status'] = pd.to_numeric(df_clean['status'], errors='coerce')
df_clean['size'] = pd.to_numeric(df_clean['size'], errors='coerce')
df_clean['file_ext'] = df_clean['file_ext'].astype('string').str.lstrip('.')
df_clean['method'] = df_clean['method'].astype('category')
df_clean['protocol'] = df_clean['protocol'].astype('category')

# Preview clean data
print("Cleaned DataFrame:")
display(df_clean.head())

Cleaned DataFrame:


,host,timestamp,request,status,size,method,url,protocol,file_ext
0,local,1994-10-24 19:41:41,GET index.html HTTP/1.0,200,150.0,GET,index.html,HTTP/1.0,html
1,local,1994-10-24 19:41:41,GET 1.gif HTTP/1.0,200,1210.0,GET,1.gif,HTTP/1.0,gif
2,local,1994-10-24 19:43:13,GET index.html HTTP/1.0,200,3185.0,GET,index.html,HTTP/1.0,html
3,local,1994-10-24 19:43:14,GET 2.gif HTTP/1.0,200,2555.0,GET,2.gif,HTTP/1.0,gif
4,local,1994-10-24 19:43:15,GET 3.gif HTTP/1.0,200,36403.0,GET,3.gif,HTTP/1.0,gif


## Part 2: Analysis Questions

### Q1: Count of total log records

In [9]:
def total_log_records(df) -> int:
    """
    Q1: Count of total log records.

    Objective:
        Determine the total number of HTTP log entries in the dataset.
        Each line in the log file represents one HTTP request.

    Returns:
        int: Total number of log entries.
    """

    # TODO: Implement logic to count log records

    return len(df)  # Placeholder return


answer1 = total_log_records(df_clean)
print("Answer 1:")
print(answer1)

Answer 1:
722270


### Q2: Count of unique hosts

In [10]:
def unique_host_count(df) -> int:
    """
    Q2: Count of unique hosts.

    Objective:
        Determine how many distinct hosts accessed the server.

    Returns:
        int: Number of unique hosts.
    """

    # TODO: Implement logic to count unique hosts
    
    return df['host'].nunique()  # Placeholder return


answer2 = unique_host_count(df_clean)
print("Answer 2:")
print(answer2)

Answer 2:
2


### Q3: Date-wise unique filename counts

In [11]:
def datewise_unique_filename_counts() -> dict[str, int]:
    """
    Q3: Date-wise unique filename counts.

    Objective:
        For each date, count the number of unique filenames that accessed the server.
        The date should be in 'dd-MMM-yyyy' format (e.g., '01-Jul-1995').

    Returns:
        dict: A dictionary mapping each date to its count of unique filenames.
              Example: {'01-Jul-1995': 123, '02-Jul-1995': 150}
    """
    
    # TODO: Implement logic for date-wise unique filename counts
    
    df = df_clean.copy()
    df['date'] = df['timestamp'].dt.strftime('%d-%b-%Y')
    return df.groupby('date')['url'].nunique().to_dict()   # Placeholder return


answer3 = datewise_unique_filename_counts()
print("Answer 3:")
# print(answer3)
print(dict(list(answer3.items())[:5]))  # Show first 5 entries

Answer 3:
{'01-Apr-1995': 407, '01-Aug-1995': 663, '01-Dec-1994': 244, '01-Feb-1995': 570, '01-Jan-1995': 82}


### Q4: Number of 404 response codes

In [12]:
def count_404_errors() -> int:
    """
    Q4: Number of 404 response codes.

    Objective:
        Count how many times the HTTP 404 Not Found status appears in the logs.

    Returns:
        int: Number of 404 errors.
    """

    # TODO: Implement logic to count 404 errors
    
    df = df_clean.copy()
    return (df['status'] == 404).sum()   # Placeholder return


answer4 = count_404_errors()
print("Answer 4:")
print(answer4)

Answer 4:
23376


### Q5: Top 15 filenames with 404 responses

In [13]:
def top_15_filenames_with_404() -> list[tuple[str, int]]:
    """
    Q5: Top 15 filenames with 404 responses.

    Objective:
        Identify which requested URLs most frequently resulted in a 404 error.
        Return the top 15 filenames sorted by frequency.

    Returns:
        list: A list of tuples (filename, count), sorted by count in descending order.
              Example: [('index.html', 200), ...]
    """

    # TODO: Implement logic to find top 15 filenames with 404

    df = df_clean.copy()
    errors_404 = df[df['status'] == 404]
    top_15 = errors_404['url'].value_counts().head(15)
    return list(top_15.items())           # Placeholder return


answer5 = top_15_filenames_with_404()
print("Answer 5:")
print(answer5)

Answer 5:
[('index.html', 4694), ('4115.html', 902), ('1611.html', 649), ('5698.xbm', 585), ('710.txt', 408), ('2002.html', 258), ('2177.gif', 193), ('10695.ps', 161), ('6555.html', 153), ('487.gif', 152), ('151.html', 149), ('488.gif', 148), ('3414.gif', 148), ('40.html', 148), ('9678.gif', 142)]


### Q6: Top 15 file extension with 404 responses

In [14]:
def top_15_ext_with_404() -> list[tuple[str, int]]:
    """
    Q6: Top 15 file extensions with 404 responses.

    Objective:
        Find which file extensions generated the most 404 errors.
        Return the top 15 sorted by number of 404s.

    Returns:
        list: A list of tuples (extension, count), sorted by count in descending order.
              Example: [('html', 45), ...]
    """

    # TODO: Implement logic to find top 15 extensions with 404
    
    if 'file_ext' not in df_clean.columns:
        df_clean['file_ext'] = df_clean['url'].str.extract(r'\.([a-zA-Z0-9]+)$')[0].str.lower()
    errors_404 = df_clean[df_clean['status'] == 404]
    ext_counts = errors_404['file_ext'].value_counts().head(15)
    return list(ext_counts.items())


answer6 = top_15_ext_with_404()
print("Answer 6:")
print(answer6)

Answer 6:
[('html', 12142), ('gif', 7202), ('xbm', 824), ('ps', 754), ('jpg', 520), ('txt', 496), ('GIF', 135), ('htm', 107), ('cgi', 77), ('com', 45), ('Z', 41), ('dvi', 40), ('ca', 36), ('hmtl', 30), ('util', 29)]


### Q7: Total bandwidth transferred per day for the month of July 1995

In [15]:
def total_bandwidth_per_day() -> dict[str, int]:
    """
    Q7: Total bandwidth transferred per day for the month of July 1995.

    Objective:
        Sum the number of bytes transferred per day.
        Skip entries where the byte field is missing or '-'.

    Returns:
        dict: A dictionary mapping each date to total bytes transferred.
              Example: {'01-Jul-1995': 123456789, ...}
    """

    # TODO: Implement logic to compute total bandwidth per day
    
    july_data = df_clean[df_clean['timestamp'].dt.strftime('%b-%Y') == 'Jul-1995']
    july_data = july_data.dropna(subset=['size'])
    july_data['date_str'] = july_data['timestamp'].dt.strftime('%d-%b-%Y')
    bandwidth_by_day = july_data.groupby('date_str')['size'].sum().to_dict()
    return bandwidth_by_day    # Placeholder return


answer7 = total_bandwidth_per_day()
print("Answer 7:")
print(answer7)

Answer 7:
{'01-Jul-1995': 16986893.0, '02-Jul-1995': 7892436.0, '03-Jul-1995': 11739190.0, '04-Jul-1995': 24976177.0, '05-Jul-1995': 22468066.0, '06-Jul-1995': 20419373.0, '07-Jul-1995': 9566244.0, '08-Jul-1995': 5475250.0, '09-Jul-1995': 4312672.0, '10-Jul-1995': 13195178.0, '11-Jul-1995': 22694805.0, '12-Jul-1995': 17861622.0, '13-Jul-1995': 15959344.0, '14-Jul-1995': 16143956.0, '15-Jul-1995': 17900110.0, '16-Jul-1995': 8086988.0, '17-Jul-1995': 18423405.0, '18-Jul-1995': 17947142.0, '19-Jul-1995': 16164044.0, '20-Jul-1995': 25504026.0, '21-Jul-1995': 25910651.0, '22-Jul-1995': 6224677.0, '23-Jul-1995': 10089651.0, '24-Jul-1995': 20554069.0, '25-Jul-1995': 23269918.0, '26-Jul-1995': 26191814.0, '27-Jul-1995': 22953465.0, '28-Jul-1995': 37450120.0, '29-Jul-1995': 16285535.0, '30-Jul-1995': 21147546.0, '31-Jul-1995': 29820800.0}


### Q8: Hourly request distribution

In [16]:
def hourly_request_distribution() -> dict[int, int]:
    """
    Q8: Hourly request distribution.

    Objective:
        Count the number of requests made during each hour (00 to 23).
        Useful for understanding traffic peaks.

    Returns:
        dict: A dictionary mapping hour (int) to request count.
              Example: {0: 120, 1: 90, ..., 23: 80}
    """

    # TODO: Implement logic for hourly distribution

    hourly_data = df_clean.dropna(subset=['timestamp'])
    hourly_data['hour'] = hourly_data['timestamp'].dt.hour
    hour_counts = hourly_data['hour'].value_counts().sort_index().to_dict()
    return hour_counts     # Placeholder return


answer8 = hourly_request_distribution()
print("Answer 8:")
print(answer8)

Answer 8:
{0: 39503, 1: 32528, 2: 30632, 3: 28093, 4: 25919, 5: 22756, 6: 19705, 7: 16927, 8: 13811, 9: 11382, 10: 10527, 11: 10366, 12: 12261, 13: 15146, 14: 22027, 15: 30859, 16: 37909, 17: 46218, 18: 45658, 19: 49932, 20: 50954, 21: 52779, 22: 50316, 23: 46062}


### Q9: Top 10 most requested filenames

In [17]:
def top_10_most_requested_filenames() -> list[tuple[str, int]]:
    """
    Q9: Top 10 most requested filenames.

    Objective:
        Identify the most commonly requested URLs (irrespective of status code).

    Returns:
        list: A list of tuples (filename, count), sorted by count in descending order.
                Example: [('index.html', 500), ...]
    """

    # TODO: Implement logic to find top 10 most requested filenames
    
    df_clean['filename'] = df_clean['url'].str.extract(r'/([^/?#]+)(?:[?#]|$)', expand=False)
    valid_files = df_clean.dropna(subset=['filename'])
    top_10_files = valid_files['filename'].value_counts().head(10)
    return list(top_10_files.items()) # Placeholder return 


answer9 = top_10_most_requested_filenames()
print("Answer 9:")
print(answer9)

Answer 9:
[('search', 15), ('451', 14), ('~mildred', 10), ('547', 5), ('A><BR>', 4), ('~hyatt', 4), ('OA_presentation', 4), ('groupkit', 3), ('Policy', 3), ('GeneralAgenda', 3)]


### Q10: HTTP response code distribution

In [18]:
def response_code_distribution() -> dict[int, int]:
    """
    Q10: HTTP response code distribution.

    Objective:
        Count how often each HTTP status code appears in the logs.

    Returns:
        dict: A dictionary mapping HTTP status codes (as int) to their frequency.
              Example: {200: 150000, 404: 3000}
    """

    # TODO: Implement logic for response code counts
    
    valid_status = df_clean.dropna(subset=['status'])
    status_counts = valid_status['status'].value_counts().sort_index()
    return status_counts.to_dict()    # Placeholder return


answer10 = response_code_distribution()
print("Answer 10:")
print(answer10)

Answer 10:
{200: 566192, 302: 30260, 304: 97560, 400: 13, 401: 46, 403: 4738, 404: 23376, 500: 42, 501: 43}
